In [1]:
import pandas as pd 
import numpy as np 
from os.path import join
import re

In [2]:
path = '/work/mflora/LSRS/'

In [3]:
names = ['StormEvents_details-ftp_v1.0_d2017_c20210803.csv',
         'StormEvents_details-ftp_v1.0_d2018_c20210803.csv',
         'StormEvents_details-ftp_v1.0_d2019_c20210803.csv',
         'StormEvents_details-ftp_v1.0_d2020_c20220322.csv'
        ]

dtype = {'VALID': np.int64, 'LAT':np.float64, 'LON':np.float64, 'BEGIN_YEARMONTH':object, 
         'BEGIN_DAY' : object, 'BEGIN_TIME' : object,
         'MAG':np.float64, 'EVENT_TYPE':object, 'TOR_F_SCALE' :object}
            
cols = ['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 
        'BEGIN_LAT', 'BEGIN_LON', 'MAGNITUDE', 'EVENT_TYPE', 'TOR_F_SCALE', 'CZ_TIMEZONE']

dfs = []
for name in names:
    df = pd.read_csv(join(path,name), usecols=cols, dtype=dtype, na_values=None)
    
    # Format the dates and times and create the VALIDTIME
    dates = df['BEGIN_DAY']
    dates = [f'{(int(d)):02d}' for d in dates]

    times = df['BEGIN_TIME']
    new_times = []
    for t in times:
        if len(t)==1:
            t = f'000{t}'
        elif len(t)==2:
            t = f'00{t}'
        elif len(t)==3:
            t = f'0{t}'
        new_times.append(t)

    df['VALID'] = df['BEGIN_YEARMONTH']+dates+new_times
    df = df.rename({'MAGNITUDE' : 'MAG', 
           'BEGIN_LAT' : 'LAT', 
           'BEGIN_LON' : 'LON',
          }, 
          axis='columns')

    df = df.drop(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME'], axis='columns')
    dfs.append(df)
    
combined_df = pd.concat(dfs)   

# Only keep the severe wx events. 
df = combined_df[combined_df['EVENT_TYPE'].isin(['Thunderstorm Wind', 'Tornado', 'Hail', 'Flood'])]

new_df = df.copy()

# Convert from local time to UTC. 
time_zone = df['CZ_TIMEZONE']
time_zone = [int(re.findall(r'\d+', t)[0]) for t in time_zone]

date = pd.to_datetime(df.VALID.astype(str), format='%Y%m%d%H%M')
hrs = pd.to_timedelta(time_zone, 'h')

new_date = date + hrs
new_df['VALID'] = new_date.dt.strftime('%Y%m%d%H%M')

In [4]:
new_df.to_csv(join(path,'STORM_DATA_2017-2020.csv'))

In [5]:
new_df['TOR_F_SCALE']

,EVENT_TYPE,CZ_TIMEZONE,MAG,TOR_F_SCALE,LAT,LON,VALID
0,Thunderstorm Wind,EST-5,52.0,NaN,39.6600,-75.0800,201704062009
1,Tornado,EST-5,NaN,EF0,26.5010,-81.9980,201704061430
2,Thunderstorm Wind,EST-5,50.0,NaN,39.8500,-83.9900,201704052249
3,Flood,EST-5,NaN,NaN,39.1065,-84.2875,201704162259
4,Hail,CST-6,1.5,NaN,40.9800,-95.8900,201704152150
...,...,...,...,...,...,...,...
61209,Tornado,EST-5,NaN,EF1,33.3158,-81.3944,202004130943
61210,Thunderstorm Wind,MST-7,50.0,NaN,33.3000,-111.6600,202009090005
61213,Thunderstorm Wind,EST-5,60.0,NaN,31.0800,-83.1900,202004231726
61214,Thunderstorm Wind,CST-6,65.0,NaN,31.3900,-85.4300,202004200437
